In [2]:
import os
import gym
import slimevolleygym
import numpy as np

from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback
from stable_baselines.bench import Monitor

from model import BnnPolicy

In [31]:
import gym
import slimevolleygym
import tqdm.notebook as tqdm

def evaluate(dnn, bnn, n=100):
    env = gym.make("SlimeVolley-v0")
    dnn_scores=[]
    bnn_scores=[]
    
    for i in tqdm.tqdm(range(n)):
        obs1 = env.reset()
        obs2 = obs1 # both sides always see the same initial observation.

        done = False
        total_reward = 0

        while not done:

            action1, _ = dnn.predict(obs1)
            action2, _ = bnn.predict(obs2)

            obs1, reward, done, info = env.step(action1, action2) # extra argument
            obs2 = info['otherObs']

            total_reward += reward
            # env.render()
        dnn_scores.append(total_reward)
        bnn_scores.append(-total_reward)
        
    dnn_score = np.mean(dnn_scores)
    bnn_score = np.mean(bnn_scores)
    
    dnn_var = np.std(dnn_scores)
    bnn_var = np.std(bnn_scores)
    
    print(f"DNN: {dnn_score} +- {dnn_var:.3f}")
    print(f"BNN: {bnn_score} +- {bnn_var:.3f}")
        
    return dnn_score, bnn_score, dnn_var, bnn_var

In [ ]:
dnn = PPO1.load(os.path.join("exp/self/ppo-dnn", "final_model"))
bnn = PPO1.load(os.path.join("exp/self/ppo-bnn", "final_model"))

In [33]:
dnn_score, bnn_score, dnn_var, bnn_var = evaluate(dnn, bnn, n=1000)


DNN: -4.031 += 1.279
BNN: 4.031 += 1.279
